<a href="https://colab.research.google.com/github/Edudeiko/DS-Unit-1-Sprint-1-Dealing-With-Data/blob/master/module1-statistics-probability-and-inference/Evgenii_Dudeiko_DSPT3_131_Statistics_Probability_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>
<br></br>

## *Data Science Unit 1 Sprint 3 Assignment 1*

# Apply the t-test to real data

Your assignment is to determine which issues have "statistically significant" differences between political parties in this [1980s congressional voting data](https://archive.ics.uci.edu/ml/datasets/Congressional+Voting+Records). The data consists of 435 instances (one for each congressperson), a class (democrat or republican), and 16 binary attributes (yes or no for voting for or against certain issues). Be aware - there are missing values!

Your goals:

1. Load and clean the data (or determine the best method to drop observations when running tests)
2. Using hypothesis testing, find an issue that democrats support more than republicans with p < 0.01
3. Using hypothesis testing, find an issue that republicans support more than democrats with p < 0.01
4. Using hypothesis testing, find an issue where the difference between republicans and democrats has p > 0.1 (i.e. there may not be much of a difference)

Note that this data will involve *2 sample* t-tests, because you're comparing averages across two groups (republicans and democrats) rather than a single group against a null hypothesis.

Stretch goals:

1. Refactor your code into functions so it's easy to rerun with arbitrary variables
2. Apply hypothesis testing to your personal project data (for the purposes of this notebook you can type a summary of the hypothesis you formed and tested)

In [1]:
cv_data_url = 'https://archive.ics.uci.edu/ml/machine-learning-databases/voting-records/house-votes-84.data'

In [2]:
import pandas as pd
cv_data = pd.read_csv(cv_data_url)

In [3]:
import numpy as np

In [4]:
cv_data.head()

,republican,n,y,n.1,y.1,y.2,y.3,n.2,n.3,n.4,y.4,?,y.5,y.6,y.7,n.5,y.8
0,republican,n,y,n,y,y,y,n,n,n,n,n,y,y,y,n,?
1,democrat,?,y,y,?,y,y,n,n,n,n,y,n,y,y,n,n
2,democrat,n,y,y,n,?,y,n,n,n,n,y,n,y,n,n,y
3,democrat,y,y,y,n,y,y,n,n,n,n,y,?,y,y,y,y
4,democrat,n,y,y,n,y,y,n,n,n,n,n,n,y,y,y,y


In [5]:
cv_data.count()

republican    434
n             434
y             434
n.1           434
y.1           434
y.2           434
y.3           434
n.2           434
n.3           434
n.4           434
y.4           434
?             434
y.5           434
y.6           434
y.7           434
n.5           434
y.8           434
dtype: int64

In [6]:
# Let's clean the data now
cv_data = pd.read_csv(cv_data_url, header=None)
cv_data.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16
0,republican,n,y,n,y,y,y,n,n,n,y,?,y,y,y,n,y
1,republican,n,y,n,y,y,y,n,n,n,n,n,y,y,y,n,?
2,democrat,?,y,y,?,y,y,n,n,n,n,y,n,y,y,n,n
3,democrat,n,y,y,n,?,y,n,n,n,n,y,n,y,n,n,y
4,democrat,y,y,y,n,y,y,n,n,n,n,y,?,y,y,y,y


In [7]:
cv_data.shape

(435, 17)

In [8]:
column_headers = ['Class Name', 'handicapped-infants', 
                  'water-project-cost-sharing', 
                  'adoption-of-the-budget-resolution', 'physician-fee-freeze', 
                  'el-salvador-aid', 'religious-groups-in-schools', 
                  'anti-satellite-test-ban', 'aid-to-nicaraguan-contras', 
                  'mx-missile', 'immigration', 'synfuels-corporation-cutback',
                  'education-spending', 'superfund-right-to-sue', 'crime',
                  'duty-free-exports', 'export-administration-act-south-africa']

df = pd.read_csv(cv_data_url, names=column_headers)
print(df.shape)
df.head()

(435, 17)


,Class Name,handicapped-infants,water-project-cost-sharing,adoption-of-the-budget-resolution,physician-fee-freeze,el-salvador-aid,religious-groups-in-schools,anti-satellite-test-ban,aid-to-nicaraguan-contras,mx-missile,immigration,synfuels-corporation-cutback,education-spending,superfund-right-to-sue,crime,duty-free-exports,export-administration-act-south-africa
0,republican,n,y,n,y,y,y,n,n,n,y,?,y,y,y,n,y
1,republican,n,y,n,y,y,y,n,n,n,n,n,y,y,y,n,?
2,democrat,?,y,y,?,y,y,n,n,n,n,y,n,y,y,n,n
3,democrat,n,y,y,n,?,y,n,n,n,n,y,n,y,n,n,y
4,democrat,y,y,y,n,y,y,n,n,n,n,y,?,y,y,y,y


In [9]:
df['Class Name'].unique()

array(['republican', 'democrat'], dtype=object)

In [10]:
df.set_index('Class Name')

,handicapped-infants,water-project-cost-sharing,adoption-of-the-budget-resolution,physician-fee-freeze,el-salvador-aid,religious-groups-in-schools,anti-satellite-test-ban,aid-to-nicaraguan-contras,mx-missile,immigration,synfuels-corporation-cutback,education-spending,superfund-right-to-sue,crime,duty-free-exports,export-administration-act-south-africa
Class Name,,,,,,,,,,,,,,,,
republican,n,y,n,y,y,y,n,n,n,y,?,y,y,y,n,y
republican,n,y,n,y,y,y,n,n,n,n,n,y,y,y,n,?
democrat,?,y,y,?,y,y,n,n,n,n,y,n,y,y,n,n
democrat,n,y,y,n,?,y,n,n,n,n,y,n,y,n,n,y
democrat,y,y,y,n,y,y,n,n,n,n,y,?,y,y,y,y
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
republican,n,n,y,y,y,y,n,n,y,y,n,y,y,y,n,y
democrat,n,n,y,n,n,n,y,y,y,y,n,n,n,n,n,y
republican,n,?,n,y,y,y,n,n,n,n,y,y,y,y,n,y


In [11]:
df.shape

(435, 17)

In [14]:
# It is a good idea to check the dataset for any missing values
df.isnull().sum()

Class Name                                0
handicapped-infants                       0
water-project-cost-sharing                0
adoption-of-the-budget-resolution         0
physician-fee-freeze                      0
el-salvador-aid                           0
religious-groups-in-schools               0
anti-satellite-test-ban                   0
aid-to-nicaraguan-contras                 0
mx-missile                                0
immigration                               0
synfuels-corporation-cutback              0
education-spending                        0
superfund-right-to-sue                    0
crime                                     0
duty-free-exports                         0
export-administration-act-south-africa    0
dtype: int64

In [15]:
# Deal with '?'
df = df.replace(to_replace='?', value=np.NaN)

In [16]:
df.isnull().sum()

Class Name                                  0
handicapped-infants                        12
water-project-cost-sharing                 48
adoption-of-the-budget-resolution          11
physician-fee-freeze                       11
el-salvador-aid                            15
religious-groups-in-schools                11
anti-satellite-test-ban                    14
aid-to-nicaraguan-contras                  15
mx-missile                                 22
immigration                                 7
synfuels-corporation-cutback               21
education-spending                         31
superfund-right-to-sue                     25
crime                                      17
duty-free-exports                          28
export-administration-act-south-africa    104
dtype: int64

In [18]:
# You can use the value_counts() to check on voting count
df['adoption-of-the-budget-resolution'].value_counts()

y    253
n    171
Name: adoption-of-the-budget-resolution, dtype: int64

In [19]:
# replace yes, no with 1 and 0 values
df = df.replace(('y', 'n'), (1, 0)) # or True False

#### Create two datasets: republican and democrat

In [23]:
df_rep = df[df['Class Name'] == 'republican']

In [24]:
df_dem = df[df['Class Name'] == 'democrat']

In [25]:
df_rep.shape, df_dem.shape

((168, 17), (267, 17))

In [26]:
# Now we have our clean and ready to use data
df_rep = df_rep.drop(columns='Class Name')
df_dem = df_dem.drop(columns='Class Name')
df_rep.shape, df_dem.shape

((168, 16), (267, 16))

### Using hypothesis testing, find an issue that democrats support more than republicans with p < 0.01

In [28]:
df_dem.describe().T

,count,mean,std,min,25%,50%,75%,max
handicapped-infants,258.0,0.604651,0.489876,0.0,0.0,1.0,1.0,1.0
water-project-cost-sharing,239.0,0.502092,0.501045,0.0,0.0,1.0,1.0,1.0
adoption-of-the-budget-resolution,260.0,0.888462,0.315405,0.0,1.0,1.0,1.0,1.0
physician-fee-freeze,259.0,0.054054,0.226562,0.0,0.0,0.0,0.0,1.0
el-salvador-aid,255.0,0.215686,0.412106,0.0,0.0,0.0,0.0,1.0
religious-groups-in-schools,258.0,0.476744,0.500430,0.0,0.0,0.0,1.0,1.0
anti-satellite-test-ban,259.0,0.772201,0.420224,0.0,1.0,1.0,1.0,1.0
aid-to-nicaraguan-contras,263.0,0.828897,0.377317,0.0,1.0,1.0,1.0,1.0
mx-missile,248.0,0.758065,0.429121,0.0,1.0,1.0,1.0,1.0
immigration,263.0,0.471483,0.500138,0.0,0.0,0.0,1.0,1.0


In [30]:
df_rep.describe().T

,count,mean,std,min,25%,50%,75%,max
handicapped-infants,165.0,0.187879,0.391804,0.0,0.0,0.0,0.0,1.0
water-project-cost-sharing,148.0,0.506757,0.501652,0.0,0.0,1.0,1.0,1.0
adoption-of-the-budget-resolution,164.0,0.134146,0.341853,0.0,0.0,0.0,0.0,1.0
physician-fee-freeze,165.0,0.987879,0.109760,0.0,1.0,1.0,1.0,1.0
el-salvador-aid,165.0,0.951515,0.215442,0.0,1.0,1.0,1.0,1.0
religious-groups-in-schools,166.0,0.897590,0.304104,0.0,1.0,1.0,1.0,1.0
anti-satellite-test-ban,162.0,0.240741,0.428859,0.0,0.0,0.0,0.0,1.0
aid-to-nicaraguan-contras,157.0,0.152866,0.361010,0.0,0.0,0.0,0.0,1.0
mx-missile,165.0,0.115152,0.320176,0.0,0.0,0.0,0.0,1.0
immigration,165.0,0.557576,0.498186,0.0,0.0,1.0,1.0,1.0


In [31]:
# handicapped-infants democrats support more than republicans 
# Now I need to prove it with p < 0.01
# From mean value we can democrats has more Yes

In [32]:
df_dem['handicapped-infants'].value_counts()

1.0    156
0.0    102
Name: handicapped-infants, dtype: int64

In [33]:
df_rep['handicapped-infants'].value_counts()


0.0    134
1.0     31
Name: handicapped-infants, dtype: int64

In [34]:
df_dem_mean = df_dem['handicapped-infants'].mean()
df_dem_mean

0.6046511627906976

In [35]:
df_rep_mean = df_rep['handicapped-infants'].mean()
df_rep_mean

0.18787878787878787

In [36]:
from scipy.stats import ttest_ind, ttest_ind_from_stats, ttest_rel

In [37]:
ttest_ind(df_dem['handicapped-infants'], df_rep['handicapped-infants'], nan_policy='omit')

Ttest_indResult(statistic=9.205264294809222, pvalue=1.613440327937243e-18)

In [39]:
print(f"Standard Deviation: {df_rep.std()}")

Standard Deviation: handicapped-infants                       0.391804
water-project-cost-sharing                0.501652
adoption-of-the-budget-resolution         0.341853
physician-fee-freeze                      0.109760
el-salvador-aid                           0.215442
religious-groups-in-schools               0.304104
anti-satellite-test-ban                   0.428859
aid-to-nicaraguan-contras                 0.361010
mx-missile                                0.320176
immigration                               0.498186
synfuels-corporation-cutback              0.339643
education-spending                        0.336322
superfund-right-to-sue                    0.347298
crime                                     0.135649
duty-free-exports                         0.286735
export-administration-act-south-africa    0.476168
dtype: float64


In [40]:
print(f"Mean: {df_rep.mean()}")

Mean: handicapped-infants                       0.187879
water-project-cost-sharing                0.506757
adoption-of-the-budget-resolution         0.134146
physician-fee-freeze                      0.987879
el-salvador-aid                           0.951515
religious-groups-in-schools               0.897590
anti-satellite-test-ban                   0.240741
aid-to-nicaraguan-contras                 0.152866
mx-missile                                0.115152
immigration                               0.557576
synfuels-corporation-cutback              0.132075
education-spending                        0.870968
superfund-right-to-sue                    0.860759
crime                                     0.981366
duty-free-exports                         0.089744
export-administration-act-south-africa    0.657534
dtype: float64


In [42]:
print(f"Standard Deviation: {df_dem.std()}")

Standard Deviation: handicapped-infants                       0.489876
water-project-cost-sharing                0.501045
adoption-of-the-budget-resolution         0.315405
physician-fee-freeze                      0.226562
el-salvador-aid                           0.412106
religious-groups-in-schools               0.500430
anti-satellite-test-ban                   0.420224
aid-to-nicaraguan-contras                 0.377317
mx-missile                                0.429121
immigration                               0.500138
synfuels-corporation-cutback              0.500949
education-spending                        0.352383
superfund-right-to-sue                    0.454518
crime                                     0.477962
duty-free-exports                         0.481697
export-administration-act-south-africa    0.246956
dtype: float64


In [43]:
print(f"Variance: {df_dem.var()}")

Variance: handicapped-infants                       0.239978
water-project-cost-sharing                0.251046
adoption-of-the-budget-resolution         0.099480
physician-fee-freeze                      0.051330
el-salvador-aid                           0.169832
religious-groups-in-schools               0.250430
anti-satellite-test-ban                   0.176589
aid-to-nicaraguan-contras                 0.142368
mx-missile                                0.184145
immigration                               0.250138
synfuels-corporation-cutback              0.250950
education-spending                        0.124174
superfund-right-to-sue                    0.206586
crime                                     0.228447
duty-free-exports                         0.232032
export-administration-act-south-africa    0.060987
dtype: float64


### Using hypothesis testing, find an issue that republicans support more than democrats with p < 0.01

In [65]:
print('democrats votes: \n', df_dem['physician-fee-freeze'].value_counts())
print('\n')
print('republicans votes: \n', df_rep['physician-fee-freeze'].value_counts())

democrats votes: 
 0.0    245
1.0     14
Name: physician-fee-freeze, dtype: int64


republicans votes: 
 1.0    163
0.0      2
Name: physician-fee-freeze, dtype: int64


In [66]:
ttest_ind(df_dem['physician-fee-freeze'], df_rep['physician-fee-freeze'], nan_policy='omit')

Ttest_indResult(statistic=-49.36708157301406, pvalue=1.994262314074344e-177)

In [0]:
# Using hypothesis testing, find an issue that republicans support more than democrats with p < 0.01